In [ ]:
LS = @(x) strsplit(strip(ls(x)));

In [ ]:
outDir = 'output/';
%recons = LS('output/*/*_PET_0_*_100.mat');
recons = LS([outDir '*/*_PET_0_*subject_04*_100.mat']);
NR = length(recons)

In [ ]:
tic;
for i=1:NR
  r = recons{i};
  [grp, ~] = regexp(r, ['^' outDir '([0-9]+/)(.*)(_PET.*)(_[0-9]+)(_subject.*)(_[0-9]+\.mat)$'], 'tokens', 'match');
  grp = grp{1};
  %% grp:
  % 1: dir (realisation group)
  % 2: type (brainweb|real)
  % 3: PET|PETpsf
  % 4: realisation
  % 5: subject
  % 6: iteration & ext

  gndmat = [grp{1} grp{2} grp{3} grp{4} grp{5} '_000.mat'];
  if exist(gndmat, 'file') ~= 2
    %gndmat = dir([outDir '*/' grp{2} '_*' '_*' grp{5} '_000.mat']);  % grp{3}??
    %gndmat = gndmat(1).name;
    gndmat = LS([outDir '*/' grp{2} '_*' '_*' strrep(grp{5}, '-C_-', '-C_') '_000.mat']);
    gndmat = gndmat{1};
  end
  oldmat = [outDir 'reconAPIRL_' grp{2} grp{5} '.mat'];  % strip _PET(psf)_*_, _000.mat
  %newmat = [outDir grp{1} 'reconAPIRL_' grp{2} grp{4} grp{5} '.mat'];
  newmat = [outDir grp{1} 'reconAPIRL_' grp{2} grp{5} '.mat'];
  if exist(newmat, 'file') == 2
    disp(['already existing newmat: ' newmat])
    continue
  end
  mlemats = LS([outDir grp{1} grp{2} '_PET_*'    grp{5} '_100.mat']);
  psfmats = LS([outDir grp{1} grp{2} '_PETpsf_*' grp{5} '_300.mat']);
  nReals = length(mlemats);
  if (nReals ~= length(psfmats)) || (nReals < 1)
    disp('cannot find recons')
    disp([outDir grp{1} grp{2} '_PET_*'    grp{5} '_100.mat'])
    disp([outDir grp{1} grp{2} '_PETpsf_*' grp{5} '_300.mat'])
    continue
  end

  %{
  disp(['gndmat: ' gndmat])
  disp(['oldmat: ' oldmat])
  disp(['newmat: ' newmat])
  disp(['mlemats: ' mlemats])
  disp(['psfmats: ' psfmats])
  continue
  %}

  %disp(['loading: ' oldmat])
  reconAPIRL = load(oldmat);
  reconAPIRL = reconAPIRL.reconAPIRL;
  %disp(['loading: ' gndmat])
  f = load(gndmat);
  reconAPIRL.scale_factor = f.scale_factor;
  reconAPIRL.PET = f.tAct;
  reconAPIRL.T1 = f.T1;
  reconAPIRL.Mu = f.tMu;
  %recons
  mlem = [];
  mlem_psf = [];
  for fi=1:nReals
    %disp(['loading mlemat: ' mlemats{f}])
    f = load(mlemats{fi});
    mlem = cat(4, mlem, f.Img);
    %disp(['loading psfmat: ' psfmats{f}])
    f = load(psfmats{fi});
    mlem_psf = cat(4, mlem_psf, f.Img);
  end
  reconAPIRL.mlem = permute(mlem, [4 1 2 3]);
  reconAPIRL.mlem_psf = permute(mlem_psf, [4 1 2 3]);
  save(newmat, 'reconAPIRL', '-v7.3');
  elapsed = toc;
  rate = elapsed / i;
  fprintf('%d/%d [%.3f<%.3fs, %.3fs/iter]\n', i, NR, elapsed, (NR-i) * rate, rate)
end